In [2]:
import pandas as pd

# Read in the data
crop_country = pd.read_csv('..\data-clean\\N_use_crop_country.csv')

crop_country.head()

,country,iso3_code,region_ifa,year,crop,crop_area_k_ha,n_k_t,aver_n_rate_kg_ha,fertilizer_emission_id,ghg_type_id,reliability_score,academic_citation,created_at
0,Argentina,ARG,Latin America,2018,Fruit primary and citrus fruits and treenuts,450.000,24.082438,53.516528,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN
1,Argentina,ARG,Latin America,2018,Grassland,17300.000,81.650336,4.719673,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN
2,Argentina,ARG,Latin America,2018,Maize,9039.594,439.956488,48.669939,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN
3,Argentina,ARG,Latin America,2018,Other cereals,1875.349,86.773543,46.270610,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN
4,Argentina,ARG,Latin America,2018,Residual,4947.668,51.220470,10.352447,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN


In [40]:
# read in emission factors

emission_factors = pd.read_csv('..\data\\emission-factors.csv', encoding = 'latin-1')

emission_factors

# show the row of the country "global"


,COD,Agri-Footprint Region,Country,IPCC Climate Koppen,DE EF,DE EF Level,IE Volitisation Factor IPCC,IE EF Volatilization and redeposition,IE Leaching Factor IPCC,IE EF Leaching and runoff,EF urea IPCC,EF transport (global) (kgN/ kgCO2),EF production (global) (kgN/ kgCO2)
0,1,Middle East,Afghanistan,dry,0.009,Global,0.11,0.005,0.00,0.011,0.2,0.277,3.73
1,2,Europe,Albania,wet,0.007,Regional,0.11,0.014,0.24,0.011,0.2,0.277,3.73
2,3,Africa,Algeria,dry,0.003,Regional,0.11,0.005,0.00,0.011,0.2,0.277,3.73
3,4,Oceania,American_Samoa,wet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277,3.73
4,5,Europe,Andorra,wet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277,3.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,228,Oceania,Wallis and Futuna Islands,wet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277,3.73
228,229,Africa,Western Sahara,dry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277,3.73
229,230,Middle East,Yemen,dry,-0.032,Global,0.11,0.005,0.00,0.011,0.2,0.277,3.73
230,231,Africa,Zambia,wet,-0.032,Regional,0.11,0.014,0.24,0.011,0.2,0.277,3.73


In [19]:
# create ISO3 code for each country in the emission factors dataset
iso_codes = pd.read_csv('..\data-clean\\country_iso3.csv')


# create a new column in the emission factors dataset with the ISO3 code
emission_factors['ISO3'] = emission_factors['Country'].map(iso_codes.set_index('country')['iso3'])

emission_factors.head()

# print the countries that without an ISO3 code
emission_factors[emission_factors['ISO3'].isnull()]['Country'].unique()

array([], dtype=object)

In [39]:
# create a new dataframe with the columns "crop", "country", "iso3" "direct emissions", "indirect emissions", "production emissions", "transport emissions", "urea emissions")
crop_country_emissions = crop_country[['crop', 'country', 'iso3_code','aver_n_rate_kg_ha']]

crop_country_emissions = crop_country_emissions.merge(emission_factors, left_on = 'iso3_code', right_on = 'ISO3', how = 'left')

# create the column "direct emissions" with the following formula: crop_country["aver_n_rate_kg_ha"] * emission_factor["DE EF"] * 44/28 * 265 where the emission factor is the one for the country (if available) or the global one
crop_country_emissions['direct_emissions'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['DE EF'] * 44/28 * 265

crop_country_emissions['indirect_emissions_VR'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['IE EF Volatilization and redeposition'] * 44/28 * 265

crop_country_emissions['indirect_emissions_LR'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['IE EF Leaching and runoff'] * 44/28 * 265

crop_country_emissions['indirect_emissions_total'] = crop_country_emissions['indirect_emissions_VR'] + crop_country_emissions['indirect_emissions_LR']

crop_country_emissions['production_emissions'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['EF production (global) (kgN/ kgCO2)']

crop_country_emissions['transport_emissions'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['EF transport (global) (kgN/ kgCO2)']

crop_country_emissions['urea emissions'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['EF urea IPCC'] *44/12

# create a column "total emissions" with the sum of all emissions
crop_country_emissions['total_emissions'] = crop_country_emissions['direct_emissions'] + crop_country_emissions['indirect_emissions_total'] + crop_country_emissions['production_emissions'] + crop_country_emissions['transport_emissions'] + crop_country_emissions['urea emissions']



# print the shrae of nan values for each  column
crop_country_emissions.isnull().sum()/len(crop_country_emissions)

crop                                     0.000000
country                                  0.000000
iso3_code                                0.000000
aver_n_rate_kg_ha                        0.000000
COD                                      0.000000
Agri-Footprint Region                    0.000000
Country                                  0.000000
IPCC Climate Koppen                      0.000000
DE EF                                    0.022078
DE EF Level                              0.022078
IE Volitisation Factor IPCC              0.022078
IE EF Volatilization and redeposition    0.022078
IE Leaching Factor IPCC                  0.022078
IE EF Leaching and runoff                0.022078
EF urea IPCC                             0.022078
EF transport (global) (kgN/ kgCO2)       0.000000
EF production (global) (kgN/ kgCO2)      0.000000
ISO3                                     0.000000
direct_emissions                         0.022078
indirect_emissions_VR                    0.022078
